## Importing packages and functions

In [1]:
import sqlite3
import pandas as pd
import numpy as np
import scipy.stats as stats
import matplotlib.pyplot as plt
import matplotlib as mpl
pd.set_option('display.width', 500)
pd.set_option('display.max_columns', 100)
pd.set_option('display.notebook_repr_html', True)
import seaborn as sns
sns.set_style("whitegrid")
from cycler import cycler

## Importing Data

In [173]:
# database file can be downloaded at: https://www.kaggle.com/hugomathien/soccer
# Connecting to database
database = 'database.sqlite'
conn = sqlite3.connect(database)

#Fetching data tables
player_data = pd.read_sql("SELECT * FROM Player;", conn)
player_stats_data = pd.read_sql("SELECT * FROM Player_Attributes;", conn)
team_data = pd.read_sql("SELECT * FROM Team;", conn)
match_data = pd.read_sql("SELECT * FROM Match;", conn)
conn.close()

## Preparing match_data dataframe

In [174]:
# creating dictionary with team_api_id and team_long_name
team_dict = pd.Series(team_data['team_long_name'].values,index=team_data['team_api_id']).to_dict()

# replacing team number id with team full name
match_data = match_data.replace({"home_team_api_id": team_dict});
match_data = match_data.replace({"away_team_api_id": team_dict});

# creating dictionary to replace league_id number with full name
league_dict = {1:'Belgium Jupiler League',
1729:'England Premier League',
4769:'France Ligue 1',
7809:'Germany 1. Bundesliga',
10257:'Italy Serie A',
13274:'Netherlands Eredivisie',
15722:'Poland Ekstraklasa',
17642:'Portugal Liga ZON Sagres',
19694:'Scotland Premier League',
21518:'Spain LIGA BBVA',
24558:'Switzerland Super League'}
match_data = match_data.replace({"league_id": league_dict});

## Building betting dataframe to compute average odds

In [5]:
# selecting relevant columns from match_data
selected=[2,3,4,7,8,9,10]
# selecting betting odds columns for Home, Draw and Away
for i in range(85,115,3):
    selected.append(i)
for i in range(86,115,3):
    selected.append(i)
for i in range(87,115,3):
    selected.append(i)
# creating df with selected columns
betdf = match_data.iloc[:, selected ]
betdf['avgH'] = betdf.iloc[:,7:17].mean(axis=1)
betdf['avgD'] = betdf.iloc[:,17:27].mean(axis=1)
betdf['avgA'] = betdf.iloc[:,27:37].mean(axis=1)
# dropping individual agencies columns, keeping only averages
betdf = betdf.drop(betdf.columns[7:37], axis=1) 

# create columns with expected result from average quotes and actual goal difference
exp_dict = {'avgH': 'Home','avgD':'Draw','avgA':'Away'}
# create expected outcome column by selecting lowest (average) odds 
betdf['exp_outcome'] = (betdf[betdf.columns[-3:]]).idxmin(axis=1)
betdf = betdf.replace({'exp_outcome':exp_dict})
# create actual 'outcome' column with values; 1 (Home win), 0 (Tie), -1 (Away win)
betdf['outcome'] = np.sign(betdf['home_team_goal']-betdf['away_team_goal'])
betdf.head(2)

C:\Users\petrelli\AppData\Local\Continuum\Anaconda3\lib\site-packages\ipykernel\__main__.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\petrelli\AppData\Local\Continuum\Anaconda3\lib\site-packages\ipykernel\__main__.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\petrelli\AppData\Local\Continuum\Anaconda3\lib\site-packages\ipykernel\__main__.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in th

,league_id,season,stage,home_team_api_id,away_team_api_id,home_team_goal,away_team_goal,avgH,avgD,avgA,exp_outcome,outcome
0,Belgium Jupiler League,2008/2009,1,KRC Genk,Beerschot AC,1,1,1.765556,3.322222,4.164444,Home,0
1,Belgium Jupiler League,2008/2009,1,SV Zulte-Waregem,Sporting Lokeren,0,0,1.898889,3.250000,3.616667,Home,0


In [6]:
match_data[10257:13274].head(2)

,id,country_id,league_id,season,stage,date,match_api_id,home_team_api_id,away_team_api_id,home_team_goal,away_team_goal,home_player_X1,home_player_X2,home_player_X3,home_player_X4,home_player_X5,home_player_X6,home_player_X7,home_player_X8,home_player_X9,home_player_X10,home_player_X11,away_player_X1,away_player_X2,away_player_X3,away_player_X4,away_player_X5,away_player_X6,away_player_X7,away_player_X8,away_player_X9,away_player_X10,away_player_X11,home_player_Y1,home_player_Y2,home_player_Y3,home_player_Y4,home_player_Y5,home_player_Y6,home_player_Y7,home_player_Y8,home_player_Y9,home_player_Y10,home_player_Y11,away_player_Y1,away_player_Y2,away_player_Y3,away_player_Y4,away_player_Y5,away_player_Y6,...,home_player_11,away_player_1,away_player_2,away_player_3,away_player_4,away_player_5,away_player_6,away_player_7,away_player_8,away_player_9,away_player_10,away_player_11,goal,shoton,shotoff,foulcommit,card,cross,corner,possession,B365H,B365D,B365A,BWH,BWD,BWA,IWH,IWD,IWA,LBH,LBD,LBA,PSH,PSD,PSA,WHH,WHD,WHA,SJH,SJD,SJA,VCH,VCD,VCA,GBH,GBD,GBA,BSH,BSD,BSA
10257,10258,10257,Italy Serie A,2008/2009,1,2008-08-31 00:00:00,537633,Cagliari,Lazio,1,4,1.0,2.0,4.0,6.0,8.0,3.0,5.0,7.0,4.0,6.0,5.0,1.0,2.0,4.0,6.0,8.0,2.0,4.0,6.0,8.0,4.0,6.0,1.0,3.0,3.0,3.0,3.0,6.0,6.0,6.0,8.0,8.0,11.0,1.0,3.0,3.0,3.0,3.0,7.0,...,96509.0,25735.0,30474.0,39731.0,30918.0,16351.0,39444.0,39334.0,41629.0,25594.0,25597.0,18506.0,<goal><value><comment>n</comment><stats><goals...,<shoton />,<shotoff />,<foulcommit />,<card><value><comment>y</comment><stats><ycard...,<cross />,<corner />,<possession />,3.0,3.0,2.5,2.6,2.95,2.65,2.6,2.9,2.7,2.7,2.75,2.6,NaN,NaN,NaN,2.62,2.90,2.5,2.8,3.0,2.7,3.0,2.8,2.4,2.7,3.0,2.6,2.5,3.0,2.62
10258,10259,10257,Italy Serie A,2008/2009,1,2008-08-31 00:00:00,537634,Catania,Genoa,1,0,1.0,2.0,4.0,6.0,8.0,4.0,6.0,3.0,5.0,7.0,5.0,1.0,2.0,4.0,6.0,8.0,3.0,5.0,7.0,3.0,5.0,7.0,1.0,3.0,3.0,3.0,3.0,6.0,6.0,8.0,8.0,8.0,11.0,1.0,3.0,3.0,3.0,3.0,7.0,...,37520.0,42702.0,27660.0,31214.0,41882.0,41760.0,38368.0,41757.0,27680.0,37767.0,NaN,39495.0,<goal><value><comment>n</comment><stats><goals...,<shoton />,<shotoff />,<foulcommit />,<card><value><comment>y</comment><stats><ycard...,<cross />,<corner />,<possession />,2.4,3.0,3.2,2.2,2.90,3.35,2.3,2.9,3.0,2.3,2.80,3.0,NaN,NaN,NaN,2.25,2.88,3.0,2.3,3.0,3.4,2.3,3.0,3.0,2.4,2.9,3.0,2.3,3.0,2.88


## Replacing player id with actual name

In [7]:
# creating dictionary with player_api_id and player_name
player_dict = pd.Series(player_data['player_name'].values,index=player_data['player_api_id']).to_dict()

In [8]:
for i in range(11):
    # replacing player number id with player name
    home='home_player_'+str(i+1)
    away='away_player_'+str(i+1)
    match_data[home] = (match_data[home]).map(player_dict)  
    match_data[away] = (match_data[away]).map(player_dict) 
# Using replace is too slow, switched to map
#     match_data = match_data.replace({home: player_dict});
#     match_data = match_data.replace({away: player_dict});

## Building X Dataframe for prediction

In [133]:
# creating list of selected columns from match_data
selected = list(match_data.columns)[3:5]
# selected.remove('country_id')
# selected.remove('match_api_id')
# for i in range(11):
#     # appending names of selected columns for players
#     home='home_player_'+str(i+1)
#     away='away_player_'+str(i+1)
#     selected.append(home)
#     selected.append(away)
selected

['season', 'stage']

In [167]:
# building X dataframe from selected columns
X = match_data[selected]

# modifying 'season' values into integers as in: '2008/2009' -> 2008
X['season'] = (X['season'].str.slice(0,4)).astype(int)

# using bookies averages for Home Win, Draw, and Away Win
X['avgH'] = betdf['avgH']
X['avgD'] = betdf['avgD']
X['avgA'] = betdf['avgA']
X['outcome'] = betdf['outcome']
X.dropna(how='any', inplace=True)
# del X['season']

# FEATURE 1: introducing bookies difference = max()-min() among 'avg_' odds
# avg_lst = ['avgH','avgD','avgA']
# X['diff'] = X[avg_lst].max(axis=1) - X[avg_lst].min(axis=1)
###  Basically no difference in prediction

# FEATURE 2: introducing difference between avgH and avgA 
# X['diff'] = abs(X['avgH']-X['avgA'])
###  Basically no difference in prediction

X.head(3)

C:\Users\petrelli\AppData\Local\Continuum\Anaconda3\lib\site-packages\ipykernel\__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\petrelli\AppData\Local\Continuum\Anaconda3\lib\site-packages\ipykernel\__main__.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\petrelli\AppData\Local\Continuum\Anaconda3\lib\site-packages\ipykernel\__main__.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the d

,season,stage,avgH,avgD,avgA,outcome
0,2008,1,1.765556,3.322222,4.164444,0
1,2008,1,1.898889,3.250000,3.616667,0
2,2008,1,2.462222,3.233333,2.544444,-1


In [175]:
# checking that everything is correct
print(X.describe(),'\n')
print(X.info())

             season         stage          avgH          avgD          avgA       outcome
count  22597.000000  22597.000000  22597.000000  22597.000000  22597.000000  22597.000000
mean    2011.496570     18.476169      2.585833      3.766131      4.525016      0.171217
std        2.308663     10.508960      1.704600      1.044215      3.542810      0.847156
min     2008.000000      1.000000      1.030000      1.330000      1.090000     -1.000000
25%     2009.000000      9.000000      1.675000      3.246667      2.515556     -1.000000
50%     2011.000000     18.000000      2.122222      3.401429      3.422222      0.000000
75%     2014.000000     27.000000      2.750000      3.837143      5.080000      1.000000
max     2015.000000     38.000000     29.142857     19.214286     41.666667      1.000000 

<class 'pandas.core.frame.DataFrame'>
Int64Index: 22597 entries, 0 to 24556
Data columns (total 6 columns):
season     22597 non-null int32
stage      22597 non-null int64
avgH       22597

## Logistic Regression with Cross Validation

In [169]:
from sklearn.linear_model import LogisticRegressionCV, LogisticRegression
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import accuracy_score

# breaking data into training and test
Xlr, Xtestlr, ylr, ytestlr = train_test_split(X.loc[:, X.columns != 'outcome'], X['outcome'], random_state=13)

# list of parameters
# parameters = {'C' : [0.0001, 0.001, 0.01, 0.1, 1, 10, 100, 1000]}
Cs =  [0.0001, 0.001, 0.01, 0.1, 1, 10, 100]

# calling GridSearchCV for 'C' parameter
model = LogisticRegressionCV(Cs=Cs, cv=5, scoring='accuracy', random_state=3, multi_class='multinomial')

# fitting the model
model.fit(Xlr,ylr)

# printing some results
print('Best parameter:',model.C_)
print('Scores:',model.scores_)

Best parameter: [ 0.1  0.1  0.1]
Scores: {-1: array([[ 0.52727809,  0.5260985 ,  0.5258036 ,  0.52639339,  0.52639339,
         0.52639339,  0.52639339],
       [ 0.53671483,  0.54054851,  0.5417281 ,  0.54143321,  0.54143321,
         0.54143321,  0.54143321],
       [ 0.53024491,  0.53732665,  0.53703157,  0.53732665,  0.53703157,
         0.53732665,  0.53703157],
       [ 0.52951594,  0.5298111 ,  0.53010626,  0.53010626,  0.53010626,
         0.53010626,  0.53010626],
       [ 0.53689492,  0.54309327,  0.54309327,  0.54309327,  0.54309327,
         0.54309327,  0.54309327]]), 0: array([[ 0.52727809,  0.5260985 ,  0.5258036 ,  0.52639339,  0.52639339,
         0.52639339,  0.52639339],
       [ 0.53671483,  0.54054851,  0.5417281 ,  0.54143321,  0.54143321,
         0.54143321,  0.54143321],
       [ 0.53024491,  0.53732665,  0.53703157,  0.53732665,  0.53703157,
         0.53732665,  0.53703157],
       [ 0.52951594,  0.5298111 ,  0.53010626,  0.53010626,  0.53010626,
         0.5

In [170]:
# Using best parameter, fitting on full training
best_model = LogisticRegression(C=model.C_[0])
best_model.fit(Xlr,ylr)

# computing accuracy on train and test sets
ytrain = best_model.predict(Xlr)
ytest = best_model.predict(Xtestlr)

print('Training set score: {0:.2f}%'.format(100*accuracy_score(ytrain,ylr)))
print('Test set score: {0:.2f}%'.format(100*accuracy_score(ytest,ytestlr)))

Training set score: 53.59%
Test set score: 51.49%


## Looking into predictions types and accuracy

In [171]:
# checking what kind of predictions are made in test set
labels = [1,0,-1]
for label in labels:
    Xcheck = X[X['outcome']==label]
    pred = ytest[ytest==label].shape[0] # ytrain[ytrain==label].shape[0]+
    print('Label =',label, 'is correct for {0:.2f}% of the games'.format(100*Xcheck.shape[0]/X.shape[0]), 
          'and is predicted {0:.2f}% of the times.'.format(100*pred/Xtestlr.shape[0]))

Label = 1 is correct for 45.91% of the games and is predicted 75.68% of the times.
Label = 0 is correct for 25.30% of the games and is predicted 0.00% of the times.
Label = -1 is correct for 28.79% of the games and is predicted 24.32% of the times.


In [172]:
# checking how many predictions are correct by predicted outcome in test set
for label in labels:
    pred = ((ytestlr==ytest) & (ytest == label)).sum() / (ytest == label).sum()
    print('Label =',label, 'predictions are correct {0:.2f}% of the times'.format(100*pred)) 

Label = 1 predictions are correct 52.13% of the times
Label = 0 predictions are correct nan% of the times
Label = -1 predictions are correct 49.49% of the times


C:\Users\petrelli\AppData\Local\Continuum\Anaconda3\lib\site-packages\ipykernel\__main__.py:3: RuntimeWarning: invalid value encountered in longlong_scalars
  app.launch_new_instance()
